# Alcohol and Life Expectancy

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error,r2_score

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

import warnings
warnings.filterwarnings('ignore')

In [2]:
df1=pd.read_csv("drinks.csv") #https://www.kaggle.com/datasets/thedevastator/relationship-between-alcohol-consumption-and-lif

In [3]:
df=pd.read_csv("lifeexpectancy-verbose.csv")

### EDA

In [4]:
df

,index,GhoCode,GhoDisplay,PublishStateCode,PublishStateDisplay,YearCode,YearDisplay,RegionCode,RegionDisplay,WorldBankIncomeGroupGroupCode,WorldBankIncomeGroupDisplay,CountryCode,CountryDisplay,SexCode,SexDisplay,DisplayValue,Numeric
0,0,WHOSIS_000001,Life expectancy at birth (years),PUBLISHED,Published,1990,1990,AMR,Americas,WB_LMI,Lower_middle_income,NIC,Nicaragua,MLE,Male,68,68.00000
1,1,WHOSIS_000002,Healthy life expectancy (HALE) at birth (years),PUBLISHED,Published,2012,2012,EUR,Europe,WB_HI,High_income,IRL,Ireland,MLE,Male,69,69.30000
2,2,WHOSIS_000001,Life expectancy at birth (years),PUBLISHED,Published,2000,2000,EMR,Eastern Mediterranean,WB_LI,Low_income,YEM,Yemen,BTSX,Both sexes,61,61.00000
3,3,WHOSIS_000001,Life expectancy at birth (years),PUBLISHED,Published,2000,2000,AFR,Africa,WB_LMI,Lower_middle_income,NGA,Nigeria,FMLE,Female,48,48.00000
4,4,WHOSIS_000015,Life expectancy at age 60 (years),PUBLISHED,Published,1990,1990,SEAR,South_East Asia,WB_LMI,Lower_middle_income,THA,Thailand,FMLE,Female,19,19.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6403,6403,WHOSIS_000001,Life expectancy at birth (years),PUBLISHED,Published,2000,2000,EMR,Eastern Mediterranean,WB_LMI,Lower_middle_income,MAR,Morocco,BTSX,Both sexes,68,68.00000
6404,6404,WHOSIS_000015,Life expectancy at age 60 (years),PUBLISHED,Published,1990,1990,WPR,Western Pacific,WB_UMI,Upper_middle_income,PLW,Palau,BTSX,Both sexes,16,16.00000
6405,6405,WHOSIS_000001,Life expectancy at birth (years),PUBLISHED,Published,1990,1990,AFR,Africa,WB_LI,Low_income,BFA,Burkina Faso,FMLE,Female,51,51.00000
6406,6406,WHOSIS_000002,Healthy life expectancy (HALE) at birth (years),PUBLISHED,Published,2012,2012,EUR,Europe,WB_LMI,Lower_middle_income,ALB,Albania,MLE,Male,64,63.50000


In [5]:
df1

,index,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
0,0,Afghanistan,0,0,0,0.0
1,1,Albania,89,132,54,4.9
2,2,Algeria,25,0,14,0.7
3,3,Andorra,245,138,312,12.4
4,4,Angola,217,57,45,5.9
...,...,...,...,...,...,...
188,188,Venezuela,333,100,3,7.7
189,189,Vietnam,111,2,1,2.0
190,190,Yemen,6,0,0,0.1
191,191,Zambia,32,19,4,2.5


In [6]:
df["PublishStateCode"].value_counts()

PUBLISHED    6408
Name: PublishStateCode, dtype: int64

In [7]:
df["PublishStateDisplay"].value_counts()

Published    6408
Name: PublishStateDisplay, dtype: int64

In [8]:
del df1["index"]
df=df.drop(["index","PublishStateCode","PublishStateDisplay","RegionDisplay","WorldBankIncomeGroupDisplay","CountryCode","SexDisplay","DisplayValue","YearDisplay"],axis=1)
#We delete some columns.

In [9]:
df.rename(columns ={'CountryDisplay':'country'},inplace=True) #We change the name of the CountryDisplay column to country.

In [10]:
df.isnull().sum() #We examine the empty lines.

GhoCode                             0
GhoDisplay                          0
YearCode                            0
RegionCode                          0
WorldBankIncomeGroupGroupCode    1164
country                             0
SexCode                             0
Numeric                             0
dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6408 entries, 0 to 6407
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   GhoCode                        6408 non-null   object 
 1   GhoDisplay                     6408 non-null   object 
 2   YearCode                       6408 non-null   int64  
 3   RegionCode                     6408 non-null   object 
 4   WorldBankIncomeGroupGroupCode  5244 non-null   object 
 5   country                        6408 non-null   object 
 6   SexCode                        6408 non-null   object 
 7   Numeric                        6408 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 400.6+ KB


In [12]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       193 non-null    object 
 1   beer_servings                 193 non-null    int64  
 2   spirit_servings               193 non-null    int64  
 3   wine_servings                 193 non-null    int64  
 4   total_litres_of_pure_alcohol  193 non-null    float64
dtypes: float64(1), int64(3), object(1)
memory usage: 7.7+ KB


In [13]:
df=pd.merge(df1,df) #We combine df1 and df.

In [33]:
df

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,GhoCode,GhoDisplay,YearCode,RegionCode,WorldBankIncomeGroupGroupCode,SexCode,Numeric
0,Afghanistan,0,0,0,0.0,WHOSIS_000015,0,2000,4,3,0,16.0
1,Afghanistan,0,0,0,0.0,WHOSIS_000002,1,2013,4,3,0,49.5
2,Afghanistan,0,0,0,0.0,WHOSIS_000015,0,2012,4,3,0,17.0
3,Afghanistan,0,0,0,0.0,WHOSIS_000015,0,2012,4,3,1,15.0
4,Afghanistan,0,0,0,0.0,WHOSIS_000002,1,2012,4,3,0,48.9
...,...,...,...,...,...,...,...,...,...,...,...,...
4504,Zimbabwe,64,18,4,4.7,WHOSIS_000002,1,2000,1,3,1,37.0
4505,Zimbabwe,64,18,4,4.7,WHOSIS_000001,2,2012,1,3,2,58.0
4506,Zimbabwe,64,18,4,4.7,WHOSIS_000002,1,2013,1,3,0,51.8
4507,Zimbabwe,64,18,4,4.7,WHOSIS_000001,2,1990,1,3,1,60.0


### Feature Engineering

In [15]:
df["WorldBankIncomeGroupGroupCode"].value_counts()

WB_LMI    1215
WB_HI     1215
WB_UMI    1053
WB_LI     1026
Name: WorldBankIncomeGroupGroupCode, dtype: int64

In [16]:
df=df.dropna() #We delete rows containing empty data.

In [31]:
abs(df.corr()["Numeric"].sort_values(ascending=False))

Numeric                          1.000000
GhoDisplay                       0.867701
YearCode                         0.207003
wine_servings                    0.145893
beer_servings                    0.131670
total_litres_of_pure_alcohol     0.121756
spirit_servings                  0.107618
RegionCode                       0.024287
SexCode                          0.031832
WorldBankIncomeGroupGroupCode    0.107931
Name: Numeric, dtype: float64

In [18]:
df["WorldBankIncomeGroupGroupCode"]=df["WorldBankIncomeGroupGroupCode"].map({"WB_LMI":0,"WB_HI":1,"WB_UMI":2,"WB_LI":3})
#We are editing some data in WorldBankIncomeGroupGroupCode.
df['WorldBankIncomeGroupGroupCode']=df['WorldBankIncomeGroupGroupCode'].astype(int)
#Changes the type of WorldBankIncomeGroupGroupCode to integer.

In [19]:
df["SexCode"].value_counts()

FMLE    1503
MLE     1503
BTSX    1503
Name: SexCode, dtype: int64

In [20]:
df["SexCode"]=df["SexCode"].map({"FMLE":0,"MLE":1,"BTSX":2})
df['SexCode']=df['SexCode'].astype(int)

In [27]:
df["GhoDisplay"].value_counts()

0    1503
1    1503
2    1503
Name: GhoDisplay, dtype: int64

In [22]:
df["GhoDisplay"]=df["GhoDisplay"].map({"Life expectancy at age 60 (years)":0,"Healthy life expectancy (HALE) at birth (years)":1,"Life expectancy at birth (years)":2})
df['GhoDisplay']=df['GhoDisplay'].astype(int)

In [23]:
df=df.reset_index()

In [30]:
del df["index"]

In [25]:
df["RegionCode"].value_counts()

EUR     1323
AFR     1107
AMR      729
WPR      594
EMR      513
SEAR     243
Name: RegionCode, dtype: int64

In [26]:
df["RegionCode"]=df["RegionCode"].map({"EUR":0,"AFR":1,"AMR":2,"WPR":3,"EMR":4,"SEAR":5})
df['RegionCode']=df['RegionCode'].astype(int)

### Regression

In [34]:
x,y=df.drop(["GhoCode","country","Numeric"],axis=1),df[["Numeric"]]
x=scaler.fit_transform(x)
x.shape

(4509, 9)

In [35]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [36]:
model=Sequential()
model.add(Dense(50,activation="relu"))
model.add(Dense(50,activation="relu"))
model.add(Dense(50,activation="relu"))
model.add(Dense(50,activation="relu"))
model.add(Dense(50,activation="relu"))
model.add(Dense(1))
model.compile(optimizer="adam",loss="mse")

In [37]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=10,epochs=500)
model.summary()

Epoch 1/500
361/361 [==============================] - 2s 2ms/step - loss: 448.9374 - val_loss: 77.6069
Epoch 2/500
361/361 [==============================] - 0s 1ms/step - loss: 59.7525 - val_loss: 47.6324
Epoch 3/500
361/361 [==============================] - 0s 1ms/step - loss: 42.8907 - val_loss: 35.8908
Epoch 4/500
361/361 [==============================] - 0s 1ms/step - loss: 35.5471 - val_loss: 37.0634
Epoch 5/500
361/361 [==============================] - 0s 1ms/step - loss: 31.1932 - val_loss: 27.9526
Epoch 6/500
361/361 [==============================] - 0s 1ms/step - loss: 27.8726 - val_loss: 25.5059
Epoch 7/500
361/361 [==============================] - 0s 1ms/step - loss: 26.3041 - val_loss: 32.9808
Epoch 8/500
361/361 [==============================] - 0s 1ms/step - loss: 23.3294 - val_loss: 38.0523
Epoch 9/500
361/361 [==============================] - 0s 1ms/step - loss: 22.5115 - val_loss: 21.6547
Epoch 10/500
361/361 [==============================] - 0s 1ms/step - lo

361/361 [==============================] - 0s 1ms/step - loss: 8.7672 - val_loss: 9.3465
Epoch 81/500
361/361 [==============================] - 0s 1ms/step - loss: 8.7116 - val_loss: 11.4193
Epoch 82/500
361/361 [==============================] - 0s 1ms/step - loss: 8.4176 - val_loss: 9.0892
Epoch 83/500
361/361 [==============================] - 0s 1ms/step - loss: 8.6349 - val_loss: 8.5208
Epoch 84/500
361/361 [==============================] - 0s 1ms/step - loss: 8.3236 - val_loss: 8.6374
Epoch 85/500
361/361 [==============================] - 0s 1ms/step - loss: 8.6631 - val_loss: 8.9357
Epoch 86/500
361/361 [==============================] - 0s 1ms/step - loss: 8.1590 - val_loss: 8.8741
Epoch 87/500
361/361 [==============================] - 0s 1ms/step - loss: 8.5796 - val_loss: 9.2076
Epoch 88/500
361/361 [==============================] - 0s 1ms/step - loss: 8.2654 - val_loss: 8.5389
Epoch 89/500
361/361 [==============================] - 0s 1ms/step - loss: 7.9997 - val_loss:

Epoch 160/500
361/361 [==============================] - 0s 1ms/step - loss: 4.7166 - val_loss: 5.5506
Epoch 161/500
361/361 [==============================] - 0s 1ms/step - loss: 4.5227 - val_loss: 7.1523
Epoch 162/500
361/361 [==============================] - 0s 1ms/step - loss: 4.7254 - val_loss: 7.6910
Epoch 163/500
361/361 [==============================] - 0s 1ms/step - loss: 4.7942 - val_loss: 5.6111
Epoch 164/500
361/361 [==============================] - 0s 1ms/step - loss: 4.8542 - val_loss: 5.2076
Epoch 165/500
361/361 [==============================] - 0s 1ms/step - loss: 4.6587 - val_loss: 8.6816
Epoch 166/500
361/361 [==============================] - 0s 1ms/step - loss: 4.7811 - val_loss: 5.4521
Epoch 167/500
361/361 [==============================] - 0s 1ms/step - loss: 4.9397 - val_loss: 6.4812
Epoch 168/500
361/361 [==============================] - 0s 1ms/step - loss: 4.7080 - val_loss: 5.5632
Epoch 169/500
361/361 [==============================] - 0s 1ms/step - lo

361/361 [==============================] - 0s 1ms/step - loss: 2.5812 - val_loss: 3.5699
Epoch 319/500
361/361 [==============================] - 0s 1ms/step - loss: 2.3231 - val_loss: 3.6547
Epoch 320/500
361/361 [==============================] - 0s 1ms/step - loss: 2.4071 - val_loss: 3.3786
Epoch 321/500
361/361 [==============================] - 0s 1ms/step - loss: 2.4040 - val_loss: 3.0408
Epoch 322/500
361/361 [==============================] - 0s 1ms/step - loss: 2.3727 - val_loss: 3.6997
Epoch 323/500
361/361 [==============================] - 0s 1ms/step - loss: 2.3382 - val_loss: 3.5366
Epoch 324/500
361/361 [==============================] - 0s 1ms/step - loss: 2.3434 - val_loss: 3.0090
Epoch 325/500
361/361 [==============================] - 0s 1ms/step - loss: 2.3190 - val_loss: 3.4438
Epoch 326/500
361/361 [==============================] - 0s 1ms/step - loss: 2.3371 - val_loss: 3.1498
Epoch 327/500
361/361 [==============================] - 0s 1ms/step - loss: 2.6047 - v

361/361 [==============================] - 0s 1ms/step - loss: 1.4827 - val_loss: 2.3417
Epoch 477/500
361/361 [==============================] - 0s 1ms/step - loss: 1.6878 - val_loss: 2.3163
Epoch 478/500
361/361 [==============================] - 0s 1ms/step - loss: 1.6286 - val_loss: 2.6258
Epoch 479/500
361/361 [==============================] - 0s 1ms/step - loss: 1.3693 - val_loss: 2.7648
Epoch 480/500
361/361 [==============================] - 0s 1ms/step - loss: 1.6658 - val_loss: 2.8782
Epoch 481/500
361/361 [==============================] - 0s 1ms/step - loss: 1.8421 - val_loss: 3.1848
Epoch 482/500
361/361 [==============================] - 0s 1ms/step - loss: 1.7806 - val_loss: 2.3840
Epoch 483/500
361/361 [==============================] - 0s 1ms/step - loss: 1.6993 - val_loss: 4.3829
Epoch 484/500
361/361 [==============================] - 0s 1ms/step - loss: 1.7182 - val_loss: 2.6163
Epoch 485/500
361/361 [==============================] - 0s 1ms/step - loss: 1.5705 - v

In [38]:
tahmin=model.predict(x_test)

29/29 [==============================] - 0s 784us/step


In [39]:
r2_score(tahmin,y_test)

0.9950672196370727

In [40]:
(mean_squared_error(tahmin,y_test))**0.5

1.6490052205377153